In [3]:
import json
import os
import ssl
from pathlib import Path
from urllib.request import urlopen
import requests

import pandas as pd

ssl._create_default_https_context = ssl._create_unverified_context

URL = "https://www.oecd.ai/ws/AIPO/API/dashboards/policyInitiatives.xqy?conceptUris=undefined"
OUT_DOCS_PATH = "data/oecd_docs/raw"
OUT_META_PATH = "data/oecd_docs/meta.csv"


# TODO: download metadata from https://www.oecd.ai/ws/AIPO/API/dashboards/policyInitiative.xqy?uri=http://stip.oecd.org/...
def parse_result_dict(result):
    name = result["label"]
    oecd_id = result["uri"].split("/")[-1]
    #description = result["description"]
    for field in result["fields"]:
        key = field["key"]
        value = field["value"]
        if key == "Country":
            country = value
        elif key == "Public access URL":
            document_url = value
        elif key == "Cover start date":
            start_date = value
        elif key == "Cover end date":
            end_date = value

    document_info = {
        "title": name,
     #   "description": description,
        "country": country,
        "documentUrl": document_url,
        "startDate": start_date,
        "endDate": end_date,
        "oecdId": oecd_id,
    }
    return document_info



    #dff.to_csv(OUT_META_PATH, index=False)
    #for _, (name, link) in tqdm(list(dff2[["oecdId", "documentUrl"]].iterrows())):
    #    file_path = os.path.join(OUT_DOCS_PATH, f"{name}.pdf")
    #    downloading.save_pdf_under_path(link, file_path)

In [4]:
ssl._create_default_https_context = ssl._create_unverified_context
res = requests.get(URL, verify = False)

/home/hania/.cache/pypoetry/virtualenvs/mars-auBTAh37-py3.8/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.oecd.ai'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [62]:
data =json.loads(res.text)#['results']

In [61]:
json.load(urlopen(URL))

{'metadata': {'resultCount': 631,
  'params': [{'uri': 'undefined', 'value': 'undefined'}]},
 'results': [{'uri': 'http://aipo.oecd.org/2021/data/policyInitiatives/26851',
   'label': '\u200bARTIFICIAL INTELLIGENCE INNOVATION ACTION PLAN FOR INSTITUTIONS OF HIGHER EDUCATION',
   'description': 'The action plan lays out a number of objectives designed to significantly enhance China’s cadre of AI talent and its university AI curricula by 2030.',
   'link': {'view': 'policyInitiativeDetail',
    'params': {'uri': 'http://aipo.oecd.org/2021-data-policyInitiatives-26851'}},
   'fields': [{'key': 'Cover start date',
     'value': '2018',
     'keyURI': 'http://kim.oecd.org/Model#coverFrom',
     'valueURI': None},
    {'key': 'Cover end date',
     'value': '2030',
     'keyURI': 'http://kim.oecd.org/Model#coverTo',
     'valueURI': None},
    {'key': 'Public access URL',
     'value': 'https://cset.georgetown.edu/wp-content/uploads/Notice-of-the-Ministry-of-Education-on-Issuing-the-Artifici

In [63]:
#data = json.load(urlopen(URL))

df = pd.DataFrame([parse_result_dict(result) for result in data["results"]])
dff = df[~df["documentUrl"].isna()]
dff2 = dff[~dff.documentUrl.str.endswith("pdf")]

In [64]:
gov = dff[dff.documentUrl.str.contains("gov")]

In [65]:
edu = dff[dff.documentUrl.str.contains("edu")]

In [66]:
edu.shape

(12, 6)

In [58]:
gov.shape

(167, 6)

In [54]:
len(set(edu.index.values.tolist() + gov.index.values.tolist()))

179

In [9]:
def parse_url(url):
    if "pdf" in url:
        # download pdf and parse
        return
    res = requests.get(url)
    return(res.text)

In [10]:
x = parse_url(dff["documentUrl"][2])

In [9]:
import datetime
import re
import time

import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import (ElementClickInterceptedException,
                                        NoSuchWindowException,
                                        WebDriverException)
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.select import Select
from newspaper import Article
import newspaper


class Api:
    """
    Wrapper
    """
    def __init__(self, download_dir='.', headless=True, verbose = True):
        """
        Initializes firefox driver
        @param download_dir - path to directory where downloaded files will be saved
        @param headless - if driver should start without graphical interface
        """
        self.verbose = verbose
        self.print_log("Setting up driver")
        self.log_dir = "./logs"

        profile = webdriver.FirefoxProfile()
        profile.set_preference("browser.download.folderList", 2)
        profile.set_preference("browser.download.manager.useWindow", False)
        profile.set_preference("browser.download.dir", download_dir)
        
        #tutaj dodac pdf
        mimetypes = ['application/pdf', 'application/x-pdf']
        profile.set_preference("browser.helperApps.neverAsk.saveToDisk", ','.join(mimetypes))

        options = webdriver.firefox.options.Options()
        # comment to allow firefox window
        if headless:
            options.add_argument("--headless")

        self.driver = driver = webdriver.Firefox(
            executable_path="/home/hania/mi2/ai-stretegies-papers-regulations-monitoring/geckodriver",
            firefox_profile=profile,
            options=options
        )
        
    def parse_article(self, url : str):
        try:
            self.driver.get(url)
            raw_html = self.driver.page_source
            
            article = None
        
            article = newspaper.Article(url = ' ', language='en', keep_article_html = True)
            article.set_html(raw_html)
            article.parse()
            
            # get links from article
            
            links = []
            soup = BeautifulSoup(article.article_html)
            for link in soup.findAll('a'):
                links.append(link.get('href'))
            
        
            return({'html': article.article_html, 
                  'text': article.text,
                  'authors':article.authors,
                  'title': article.title,
                  'links': list(set(links))})
        except:
            self.save_snapshot()
            raise
        
    """
    Logging
    """
    def print_log(self, *args, **kwargs):
        if self.verbose:
            print(*args, **kwargs, flush=True)
    def save_snapshot(self):
        name = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S.%f")
        self.driver.save_screenshot(self.log_dir + '/' + name + ".png")
        with open(self.log_dir + '/' + name + '.html', 'w') as html_file:
            html_file.write(self.driver.page_source)

    def close(self):
        try:
            self.driver.quit()
        except:
            self.print_log('Failed to quit driver')

    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        if exc_value is not None:
            self.save_snapshot()
        self.close()

In [10]:
api = Api(headless=False)

Setting up driver


In [11]:
#dff["documentUrl"][41]

In [12]:
#api.parse_article(dff["documentUrl"][41])

In [13]:
import json
j = {'docs':[]}

In [14]:
for url in dff["documentUrl"]:
    if "pdf" in url:
        continue
    j["docs"].append(api.parse_article(url))

FileNotFoundError: [Errno 2] No such file or directory: './logs/2021-07-16_11-40-48.326655.html'

In [18]:
for i in j["docs"]:
    print(i["text"])

1. “Digitization of the economy branches – reorganization of the RK economy traditional branches using groundbreaking technologies and possibilities, which increase labour productivity and lead to the capitalization growth.

2. “Transition to the digital state” – state infrastructure transformation to provide services for population and business, anticipating their demands.

3. “Implementation of the digital Silk Way” – development of a high speed and security infrastructure of the transfer, storage and processing of data.

4. “Evolution of the human capital assets” – transformational changes, comprising creative society formation and transition to the new realities – knowledge-based economy.

5. “Innovative ecosystem formation” – creation of the conditions for technological entrepreneurship development with stable relations between business, academic domain and state, as well as introduction of innovations into industry.
#KrattAI: the next level of development of the digital state in 